In [1]:
%load_ext autoreload
%autoreload 2

import torch.backends.cudnn
torch.backends.cudnn.benchmark = True

import argparse
from operator import itemgetter

# import helpers.aligned_printer as AP
# import helpers.testset as TS
# import test.multiscale_tester as MST

# from importlib import reload
# reload(AP)
# reload(TS)
# reload(MST)

# AlignedPrinter = AP.AlignedPrinter
# Testset = TS.Testset
# MultiscaleTester = MST.MultiscaleTester

from helpers.aligned_printer import AlignedPrinter
from helpers.testset import Testset
from test.multiscale_tester import MultiscaleTester

*** AC_NEEDS_CROP_DIM = 3000000


In [2]:

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

flags = {
    'log_dir': 'weights/', 
    'log_dates': '0306_0001', 
    'images': '/var/tmp/sauravkadavath/distorted_datasets/imagenet_r/n02823750_beer_glass/', 
    'match_filenames': None, 
    'max_imgs_per_folder': None, 
    'crop': None, 
    'names': 'L3C', 
    'overwrite_cache': False, 
    'reset_entire_cache': False, 
    'restore_itr': '-1', 
    'recursive': 'auto', 
    'sample': None, 
    'write_to_files': None, 
    'compare_theory': False, 
    'time_report': None, 
    'sort_output': 'testset'
}
flags = Struct(**flags)

testsets = [Testset(images_dir_or_image.rstrip('/'), flags.max_imgs_per_folder,
                    # Append flags.crop to ID so that it creates unique entry in cache
                    append_id=f'_crop{flags.crop}' if flags.crop else None)
            for images_dir_or_image in flags.images.split(',')]
if flags.match_filenames:
    for ts in testsets:
        ts.filter_filenames(flags.match_filenames)

splitter = ',' if ',' in flags.log_dates else '|'  # support tensorboard strings, too
results = []
log_dates = flags.log_dates.split(splitter)
for log_date in log_dates:
    for restore_itr in map(int, flags.restore_itr.split(',')):
        print('Testing {} at {} ---'.format(log_date, restore_itr))
        tester = MultiscaleTester(log_date, flags, restore_itr, deepaugment=True)
        results += tester.test_all(testsets)

print("-----")
print(results)
print("-----")

# if --names was passed: will print 'name (log_date)'. otherwise, will just print 'log_date'
if flags.names:
    names = flags.names.split(splitter) if flags.names else log_dates
    names_to_log_date = {log_date: f'{name} ({log_date})'
                         for log_date, name in zip(log_dates, names)}
else:
    # set names to log_dates if --names is not given, i.e., we just output log_date
    names_to_log_date = {log_date: log_date for log_date in log_dates}


Testing 0306_0001 at -1 ---
Restoring weights/0306_0001 cr oi/ckpts/ckpt_0001002750.pt... (strict=True)
Testing Testset(n02823750_beer_glass): in /var/tmp/sauravkadavath/distorted_datasets/imagenet_r/n02823750_beer_glass, 84 images
Running art_0
Input shape =  torch.Size([1, 3, 504, 504])
enc_out for scale =  0
torch.Size([1, 5, 252, 252])
torch.Size([1, 5, 252, 252])
torch.Size([1, 5, 252, 252])
25
torch.Size([1, 64, 252, 252])
enc_out for scale =  1
torch.Size([1, 5, 126, 126])
torch.Size([1, 5, 126, 126])
torch.Size([1, 5, 126, 126])
25
torch.Size([1, 64, 126, 126])
enc_out for scale =  2
torch.Size([1, 5, 63, 63])
torch.Size([1, 5, 63, 63])
torch.Size([1, 5, 63, 63])
25
torch.Size([1, 64, 63, 63])
torch.Size([1, 64, 504, 504])
0: P.shape =  torch.Size([1, 120, 504, 504])
1: P.shape =  torch.Size([1, 150, 252, 252])
2: P.shape =  torch.Size([1, 150, 126, 126])
------------------------
Running cartoon_0
Input shape =  torch.Size([1, 3, 504, 504])
enc_out for scale =  0
torch.Size([1,

enc_out for scale =  0
torch.Size([1, 5, 252, 184])
torch.Size([1, 5, 252, 184])
torch.Size([1, 5, 252, 184])
25
torch.Size([1, 64, 252, 184])
enc_out for scale =  1
torch.Size([1, 5, 126, 92])
torch.Size([1, 5, 126, 92])
torch.Size([1, 5, 126, 92])
25
torch.Size([1, 64, 126, 92])
enc_out for scale =  2
torch.Size([1, 5, 63, 46])
torch.Size([1, 5, 63, 46])
torch.Size([1, 5, 63, 46])
25
torch.Size([1, 64, 63, 46])
torch.Size([1, 64, 504, 368])
0: P.shape =  torch.Size([1, 120, 504, 368])
1: P.shape =  torch.Size([1, 150, 252, 184])
2: P.shape =  torch.Size([1, 150, 126, 92])
------------------------
Running cartoon_21
Input shape =  torch.Size([1, 3, 504, 352])
enc_out for scale =  0
torch.Size([1, 5, 252, 176])
torch.Size([1, 5, 252, 176])
torch.Size([1, 5, 252, 176])
25
torch.Size([1, 64, 252, 176])
enc_out for scale =  1
torch.Size([1, 5, 126, 88])
torch.Size([1, 5, 126, 88])
torch.Size([1, 5, 126, 88])
25
torch.Size([1, 64, 126, 88])
enc_out for scale =  2
torch.Size([1, 5, 63, 44])

torch.Size([1, 64, 344, 504])
0: P.shape =  torch.Size([1, 120, 344, 504])
1: P.shape =  torch.Size([1, 150, 172, 252])
2: P.shape =  torch.Size([1, 150, 86, 126])
------------------------
Running cartoon_9
Input shape =  torch.Size([1, 3, 376, 504])
enc_out for scale =  0
torch.Size([1, 5, 188, 252])
torch.Size([1, 5, 188, 252])
torch.Size([1, 5, 188, 252])
25
torch.Size([1, 64, 188, 252])
enc_out for scale =  1
torch.Size([1, 5, 94, 126])
torch.Size([1, 5, 94, 126])
torch.Size([1, 5, 94, 126])
25
torch.Size([1, 64, 94, 126])
enc_out for scale =  2
torch.Size([1, 5, 47, 63])
torch.Size([1, 5, 47, 63])
torch.Size([1, 5, 47, 63])
25
torch.Size([1, 64, 47, 63])
torch.Size([1, 64, 376, 504])
0: P.shape =  torch.Size([1, 120, 376, 504])
1: P.shape =  torch.Size([1, 150, 188, 252])
2: P.shape =  torch.Size([1, 150, 94, 126])
------------------------
Running deviantart_0
Input shape =  torch.Size([1, 3, 352, 200])
enc_out for scale =  0
torch.Size([1, 5, 176, 100])
torch.Size([1, 5, 176, 100

Input shape =  torch.Size([1, 3, 288, 504])
enc_out for scale =  0
torch.Size([1, 5, 144, 252])
torch.Size([1, 5, 144, 252])
torch.Size([1, 5, 144, 252])
25
torch.Size([1, 64, 144, 252])
enc_out for scale =  1
torch.Size([1, 5, 72, 126])
torch.Size([1, 5, 72, 126])
torch.Size([1, 5, 72, 126])
25
torch.Size([1, 64, 72, 126])
enc_out for scale =  2
torch.Size([1, 5, 36, 63])
torch.Size([1, 5, 36, 63])
torch.Size([1, 5, 36, 63])
25
torch.Size([1, 64, 36, 63])
torch.Size([1, 64, 288, 504])
0: P.shape =  torch.Size([1, 120, 288, 504])
1: P.shape =  torch.Size([1, 150, 144, 252])
2: P.shape =  torch.Size([1, 150, 72, 126])
------------------------
Running graphic_3
Input shape =  torch.Size([1, 3, 504, 504])
enc_out for scale =  0
torch.Size([1, 5, 252, 252])
torch.Size([1, 5, 252, 252])
torch.Size([1, 5, 252, 252])
25
torch.Size([1, 64, 252, 252])
enc_out for scale =  1
torch.Size([1, 5, 126, 126])
torch.Size([1, 5, 126, 126])
torch.Size([1, 5, 126, 126])
25
torch.Size([1, 64, 126, 126])
en

torch.Size([1, 64, 504, 456])
0: P.shape =  torch.Size([1, 120, 504, 456])
1: P.shape =  torch.Size([1, 150, 252, 228])
2: P.shape =  torch.Size([1, 150, 126, 114])
------------------------
Running painting_4
Input shape =  torch.Size([1, 3, 504, 304])
enc_out for scale =  0
torch.Size([1, 5, 252, 152])
torch.Size([1, 5, 252, 152])
torch.Size([1, 5, 252, 152])
25
torch.Size([1, 64, 252, 152])
enc_out for scale =  1
torch.Size([1, 5, 126, 76])
torch.Size([1, 5, 126, 76])
torch.Size([1, 5, 126, 76])
25
torch.Size([1, 64, 126, 76])
enc_out for scale =  2
torch.Size([1, 5, 63, 38])
torch.Size([1, 5, 63, 38])
torch.Size([1, 5, 63, 38])
25
torch.Size([1, 64, 63, 38])
torch.Size([1, 64, 504, 304])
0: P.shape =  torch.Size([1, 120, 504, 304])
1: P.shape =  torch.Size([1, 150, 252, 152])
2: P.shape =  torch.Size([1, 150, 126, 76])
------------------------
Running painting_5
Input shape =  torch.Size([1, 3, 504, 376])
enc_out for scale =  0
torch.Size([1, 5, 252, 188])
torch.Size([1, 5, 252, 188

Running sketch_18
Input shape =  torch.Size([1, 3, 360, 456])
enc_out for scale =  0
torch.Size([1, 5, 180, 228])
torch.Size([1, 5, 180, 228])
torch.Size([1, 5, 180, 228])
25
torch.Size([1, 64, 180, 228])
enc_out for scale =  1
torch.Size([1, 5, 90, 114])
torch.Size([1, 5, 90, 114])
torch.Size([1, 5, 90, 114])
25
torch.Size([1, 64, 90, 114])
enc_out for scale =  2
torch.Size([1, 5, 45, 57])
torch.Size([1, 5, 45, 57])
torch.Size([1, 5, 45, 57])
25
torch.Size([1, 64, 45, 57])
torch.Size([1, 64, 360, 456])
0: P.shape =  torch.Size([1, 120, 360, 456])
1: P.shape =  torch.Size([1, 150, 180, 228])
2: P.shape =  torch.Size([1, 150, 90, 114])
------------------------
Running sketch_19
Input shape =  torch.Size([1, 3, 1024, 1024])
enc_out for scale =  0
torch.Size([1, 5, 512, 512])
torch.Size([1, 5, 512, 512])
torch.Size([1, 5, 512, 512])
25
torch.Size([1, 64, 512, 512])
enc_out for scale =  1
torch.Size([1, 5, 256, 256])
torch.Size([1, 5, 256, 256])
torch.Size([1, 5, 256, 256])
25
torch.Size([

torch.Size([1, 64, 504, 416])
0: P.shape =  torch.Size([1, 120, 504, 416])
1: P.shape =  torch.Size([1, 150, 252, 208])
2: P.shape =  torch.Size([1, 150, 126, 104])
------------------------
Running sticker_1
Input shape =  torch.Size([1, 3, 464, 504])
enc_out for scale =  0
torch.Size([1, 5, 232, 252])
torch.Size([1, 5, 232, 252])
torch.Size([1, 5, 232, 252])
25
torch.Size([1, 64, 232, 252])
enc_out for scale =  1
torch.Size([1, 5, 116, 126])
torch.Size([1, 5, 116, 126])
torch.Size([1, 5, 116, 126])
25
torch.Size([1, 64, 116, 126])
enc_out for scale =  2
torch.Size([1, 5, 58, 63])
torch.Size([1, 5, 58, 63])
torch.Size([1, 5, 58, 63])
25
torch.Size([1, 64, 58, 63])
torch.Size([1, 64, 464, 504])
0: P.shape =  torch.Size([1, 120, 464, 504])
1: P.shape =  torch.Size([1, 150, 232, 252])
2: P.shape =  torch.Size([1, 150, 116, 126])
------------------------
Running sticker_2
Input shape =  torch.Size([1, 3, 504, 376])
enc_out for scale =  0
torch.Size([1, 5, 252, 188])
torch.Size([1, 5, 252, 